In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error

import numpy as np

#### this function builds a Quantum Fourier Transform gate
(this should be double with a qiskit library too, but I couldn't convert later circuits to gate that way)

In [2]:
def QuantumFourierTransform(digits: int):
    QRegY = QuantumRegister(digits, "y")
    
    QC = QuantumCircuit(QRegY)

    for i in range(len(QRegY)):
        QC.h(QRegY[i])

        theta = np.pi / 2
        for j in range(i + 1, len(QRegY)):
            QC.cp(theta, QRegY[j], QRegY[i])
            theta /= 2
    
    for i in range(digits >> 1):
        QC.swap(QRegY[i],QRegY[- (i + 1)])
    
    return QC.to_gate()

#### the following function builds a gate (oracle) that transforms $| x \rangle | y \rangle$ to $| x \rangle | y - f(x) \rangle$

In [3]:
def EnergyOracle(GraphLaplacian: list[list[int]], digits: int):

    QRegX = QuantumRegister(len(GraphLaplacian), "x")
    QRegY = QuantumRegister(digits, "y")

    QC = QuantumCircuit(QRegX, QRegY)
    
    QFT = QuantumFourierTransform(digits)
    InverseQFT = QFT.inverse()
    
    QC.append(QFT, QRegY)

    # Phasers
    for i in range(len(GraphLaplacian)):
        theta = - GraphLaplacian[i][i] * np.pi
        for j in range(len(QRegY)):
            QC.cp(theta, QRegX[i], QRegY[j])
            theta /= 2

        for j in range(i + 1, len(GraphLaplacian[i])):
            if GraphLaplacian[i][j] != 0:
                theta = - 2 * GraphLaplacian[i][j] * np.pi
                for k in range(len(QRegY)):
                    QC.mcp(theta, [QRegX[i], QRegX[j]], QRegY[k])
                    theta /= 2

    QC.append(InverseQFT, QRegY)

    return QC.to_gate()

#### the following function builds the $G(\alpha, \beta)$ gates for the fixed point search

In [4]:
def GroverBangBang(GraphLaplacian: list[list[int]], digits: int, alpha: float, beta: float):
    
    QRegX = QuantumRegister(len(GraphLaplacian))
    QRegY = QuantumRegister(digits)

    QC = QuantumCircuit(QRegX, QRegY)

    # S_t (beta)
    QC.append(EnergyOracle(GraphLaplacian, digits), QRegX[:] + QRegY[:])
    QC.p(beta, QRegY[0])
    QC.append(EnergyOracle(GraphLaplacian, digits).inverse(), QRegX[:] + QRegY[:])

    # S_s (alpha)
    QC.h(QRegX)
    QC.x(QRegX)
    QC.mcp(- alpha, QRegX[:-1], QRegX[-1])
    QC.x(QRegX)
    QC.h(QRegX)

    return QC.to_gate()

#### the following creates the $S_L$ circuit

In [5]:
def GroverFixedPointGate(GraphLaplacian: list[list[int]], digits: int, l: int, delta: float):

    gamma = np.sqrt(1 - 1 / pow(np.cosh(np.arccosh(1 / delta) / (2 * l + 1)), 2))

    alpha = []

    for j in range(1, l + 1):        
        alpha.append(2 * np.arctan(1 / (gamma * np.tan(2 * np.pi * j / (2 * l + 1)))))

    QRegX = QuantumRegister(len(GraphLaplacian), "x")
    QRegY = QuantumRegister(digits, "y")

    QC = QuantumCircuit(QRegX, QRegY)

    QC.h(QRegX)

    for j in range(l):
        QC.append(GroverBangBang(GraphLaplacian, digits, alpha[j], - alpha[l - 1 - j]), QRegX[:] + QRegY[:])

    # add one more energy oracle to see the cut values
    QC.append(EnergyOracle(GraphLaplacian, digits), QRegX[:] + QRegY[:])

    return QC.to_gate()

#### the following builds the circuit for the new QAOA

In [6]:
def GroverQAOACircuit(GraphLaplacian: list[list[int]], digits: int, y: int, delta: float, l: int, bangs: list[list[int]], ShotNumber: int):

    # one quantum/classical register pair for each vertex
    QRegX = QuantumRegister(len(GraphLaplacian), "x")
    ClRegX = ClassicalRegister(len(GraphLaplacian), "cl-x")

    # one quantum/classical register pair for each digit
    QRegY = QuantumRegister(digits, "y")
    ClRegY = ClassicalRegister(digits, "cl-y")

    QC = QuantumCircuit(QRegX, QRegY, ClRegX, ClRegY)

    GroverMixer = GroverFixedPointGate(GraphLaplacian, digits, l, delta)
    GroverMixer.label = "GroverMixer"
    InverseGroverMixer = GroverMixer.inverse()
    InverseGroverMixer.label = "GroverMixerInverse"

    for i in range(len(QRegX)):
        QC.initialize([1, 0], QRegX[i])

    for i in range(len(QRegY)):
        QC.initialize([1 - (y >> (digits - i - 1))%2, (y >> (digits - i - 1))%2], QRegY[i])

    QC.append(GroverMixer, QRegX[:] + QRegY[:])

    # the bangs / p = len(bangs) / going backward per convention
    for k in range(len(bangs) - 1, - 1, - 1):

        #this implements the phase-bang (without totterization!!!)
        for i in range(digits):
            QC.p(bangs[k][1] * (1 << (digits - 1 - i)), QRegY[i])

        # this is the Grover-bang
        QC.append(InverseGroverMixer, QRegX[:] + QRegY[:])
        QC.x(QRegX)
        QC.mcp(bangs[k][0], QRegX[:-1], QRegX[-1])
        QC.x(QRegX)
        QC.append(GroverMixer, QRegX[:] + QRegY[:])

    QC.measure(QRegX, ClRegX)
    QC.measure(QRegY, ClRegY)

    print(QC)
    
    # Example error probabilities
    p_reset = 0.03
    p_meas = 0.1
    p_gate1 = 0.05

    # QuantumError objects
    error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    error_gate2 = error_gate1.tensor(error_gate1)
    
    # Add errors to noise model
    noise_bit_flip = NoiseModel()
    noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
    noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
    noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
    noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])

    # simulation
    noisy_simulator = AerSimulator(noise_model=noise_bit_flip)
    compiled_QC = transpile(QC, noisy_simulator)    
    counts = noisy_simulator.run(compiled_QC, shots=ShotNumber).result().get_counts(compiled_QC)
    
    adjusted_counts = [[s[::-1], counts[s] * 100 / ShotNumber] for s in counts]
    adjusted_counts.sort()

    return adjusted_counts

## TESTING:

### parameters

In [7]:
NumberOfVerticies = 6
RootLambda = 1/2 # ratio of good states to all states
P_L = 0.9
ShotNumber = 10

# empty bangs list just makes it work like a Grover search circuit
bangs = []



# from the input we compute the following
delta = np.sqrt(1 - P_L)
l = int(np.ceil(np.arccosh(1 / delta) / np.arccosh(1 / np.sqrt(1 - RootLambda * RootLambda)))) >> 1

# create large, random graph
GraphLaplacian = []
for i in range(NumberOfVerticies):
    GraphLaplacian.append([])
    for j in range(i):
        GraphLaplacian[i].append(np.random.randint(- 1, 1))
        GraphLaplacian[j].append(GraphLaplacian[i][j])
    GraphLaplacian[i].append(0)

print()
print("the graph Laplacian:")
print()
for i in range(NumberOfVerticies):
    GraphLaplacian[i][i] = - np.sum(GraphLaplacian[i])
    print(GraphLaplacian[i])
print()

NumberOfEdges = 0
for i in range(len(GraphLaplacian)):
    NumberOfEdges += GraphLaplacian[i][i]
NumberOfEdges >>= 1

components = NumberOfVerticies - np.linalg.matrix_rank(np.array(GraphLaplacian))
print("# of components =",components)

if components == 1:
    y = int(NumberOfVerticies / 2 + (np.sqrt(8 * NumberOfVerticies + 1) - 9) / 8) # one less then the Erdos-Edwards bound for arbitrary graphs
else:
    y = ((2 * len(GraphLaplacian) + NumberOfEdges - 1) >> 2) - 1 # one less then the Erdos-Edwards bound for connected graphs
digits = int(np.log2(NumberOfEdges)) + 2 # (+ 1) might already be enough


the graph Laplacian:

[3, -1, 0, 0, -1, -1]
[-1, 2, 0, 0, -1, 0]
[0, 0, 1, 0, 0, -1]
[0, 0, 0, 2, -1, -1]
[-1, -1, 0, -1, 4, -1]
[-1, 0, -1, -1, -1, 4]

# of components = 1


### results

In [8]:
print()
print("cut-energy threshold =",y + 1)
print("query complexity of Grover =",l)
print("square root of lambda =",RootLambda)
print("P_L =",P_L * 100,"%")
print()

counts = GroverQAOACircuit(GraphLaplacian, digits, y, delta, l, bangs, ShotNumber)
prob = 0
for s in counts:
    cut = 0
    for i in range(len(GraphLaplacian) - 1):
        if s[0][i] == "1":
            cut += GraphLaplacian[i][i]
            for j in range(i + 1, len(GraphLaplacian)):
                if s[0][j] == "1":
                    cut += GraphLaplacian[i][j]
    if cut > y:
        prob += s[1]
    print("configuration =",s[0][:len(GraphLaplacian)],"\tcut =",cut,"\tfrequency =",s[1],"%")

print()
print("Probability of success =",prob,"%")
print()


cut-energy threshold = 3
query complexity of Grover = 2
square root of lambda = 0.5
P_L = 90.0 %

        ┌─────────────────┐┌───────────────┐┌─┐                              
   x_0: ┤ Initialize(1,0) ├┤0              ├┤M├──────────────────────────────
        ├─────────────────┤│               │└╥┘┌─┐                           
   x_1: ┤ Initialize(1,0) ├┤1              ├─╫─┤M├───────────────────────────
        ├─────────────────┤│               │ ║ └╥┘┌─┐                        
   x_2: ┤ Initialize(1,0) ├┤2              ├─╫──╫─┤M├────────────────────────
        ├─────────────────┤│               │ ║  ║ └╥┘┌─┐                     
   x_3: ┤ Initialize(1,0) ├┤3              ├─╫──╫──╫─┤M├─────────────────────
        ├─────────────────┤│               │ ║  ║  ║ └╥┘┌─┐                  
   x_4: ┤ Initialize(1,0) ├┤4              ├─╫──╫──╫──╫─┤M├──────────────────
        ├─────────────────┤│               │ ║  ║  ║  ║ └╥┘┌─┐               
   x_5: ┤ Initialize(1,0) ├┤5  GroverMixer 